In [3]:
import psycopg2 as pg
import numpy as np
import pandas as pd
import datetime
import logging

In [4]:
connection = pg.connect(database="sidewalk", user="sidewalk", password="sidewalk", host="localhost", port="5432" )

In [5]:
volunteer_accuracy = pd.read_csv('../../data/raw/accuracies-volunteer.csv')
volunteer_accuracy.head()

,condition.id,worker1,worker2,worker3,worker4,worker5,n.workers,worker.thresh,binary,remove.low.severity,...,granularity,label.type,true.pos,false.pos,true.neg,false.neg,precision,recall,specificity,f.measure
0,70,01232fef-5a19-4435-8be6-c0da3b38cabd,null,null,null,null,1,1,True,False,...,5_meter,Problem,27,26,186,17,0.5094339622641509,0.6136363636363636,0.8773584905660378,0.556701030927835
1,70,01232fef-5a19-4435-8be6-c0da3b38cabd,null,null,null,null,1,1,True,False,...,10_meter,Problem,27,15,76,12,0.6428571428571429,0.6923076923076923,0.8351648351648352,0.6666666666666666
2,70,01232fef-5a19-4435-8be6-c0da3b38cabd,null,null,null,null,1,1,True,False,...,street,Problem,17,0,0,0,1,1,null,1
3,72,9501513f-3822-4921-861e-8f1440dee102,null,null,null,null,1,1,True,False,...,5_meter,Problem,22,58,161,7,0.275,0.7586206896551724,0.7351598173515982,0.40366972477064217
4,72,9501513f-3822-4921-861e-8f1440dee102,null,null,null,null,1,1,True,False,...,10_meter,Problem,21,44,59,3,0.3230769230769231,0.875,0.5728155339805825,0.47191011235955055


In [6]:
selected_users = volunteer_accuracy[['condition.id', 'worker1']]
selected_users = selected_users.drop_duplicates()
selected_users.head()

,condition.id,worker1
0,70,01232fef-5a19-4435-8be6-c0da3b38cabd
3,72,9501513f-3822-4921-861e-8f1440dee102
6,73,32f21407-253f-46ea-a01b-55bcf4ac2113
9,74,b65c0864-7c3a-4ba7-953b-50743a2634f6
12,75,0bfed786-ce24-43f9-9c58-084ae82ad175


In [7]:
def type_of_id(x):
    if len(x) == 36:
        return 'user_id'
    if '.' in x and '-' not in x:
        return 'ip_address'
    else:
        return 'other'

selected_users['worker1.type'] = selected_users['worker1'].apply(lambda x: type_of_id(x))
selected_users.sample(n=5)

,condition.id,worker1,worker1.type
84,103,0fddfc77-b164-40cf-989d-d0006c43532a,user_id
57,92,438a3347-4863-4888-927d-6399670edd15,user_id
18,77,c744a3ea-aa20-4d40-89e4-878ca21c002e,user_id
117,116,a8786633-e13c-4f7e-abd7-e625ab2f8cbc,user_id
144,131,143.231.249.141,ip_address


In [8]:
user_id = '424c7c8c-4547-4704-bcd4-e844dc623b9c'

pd.read_sql(
'''
    SELECT *
    FROM audit_task
    WHERE user_id='%(user_id)s'
''' % locals(), connection)

,audit_task_id,amt_assignment_id,user_id,street_edge_id,task_start,task_end,completed
0,11996,None,424c7c8c-4547-4704-bcd4-e844dc623b9c,3360,2016-11-18 20:25:36.638000+00:00,2016-11-18 20:29:16.365000+00:00,True
1,11999,None,424c7c8c-4547-4704-bcd4-e844dc623b9c,11807,2016-11-18 20:25:36.638000+00:00,2016-11-18 20:35:45.751000+00:00,True
2,12000,None,424c7c8c-4547-4704-bcd4-e844dc623b9c,6216,2016-11-18 20:25:36.638000+00:00,2016-11-18 20:38:04.648000+00:00,True
3,11994,None,424c7c8c-4547-4704-bcd4-e844dc623b9c,23677,2016-11-18 20:25:34.481000+00:00,2016-11-18 20:26:31.986000+00:00,True
4,12050,None,424c7c8c-4547-4704-bcd4-e844dc623b9c,18712,2016-11-21 16:30:41.961000+00:00,2016-11-21 16:33:16.841000+00:00,True
5,12051,None,424c7c8c-4547-4704-bcd4-e844dc623b9c,9501,2016-11-21 16:30:44.249000+00:00,2016-11-21 16:36:33.870000+00:00,True
6,12059,None,424c7c8c-4547-4704-bcd4-e844dc623b9c,1391,2016-11-21 16:46:45.323000+00:00,2016-11-21 17:00:01.916000+00:00,False
7,12022,None,424c7c8c-4547-4704-bcd4-e844dc623b9c,12539,2016-11-20 21:29:50.054000+00:00,2016-11-20 21:30:42.397000+00:00,True
8,11995,None,424c7c8c-4547-4704-bcd4-e844dc623b9c,23681,2016-11-18 20:25:36.638000+00:00,2016-11-18 20:28:34.666000+00:00,True
9,12002,None,424c7c8c-4547-4704-bcd4-e844dc623b9c,19861,2016-11-18 20:25:36.638000+00:00,2016-11-18 20:42:41.894000+00:00,True


In [9]:
user_ids = selected_users[selected_users['worker1.type'] == 'user_id']



In [27]:
def get_audit_tasks(user_id):
    return pd.read_sql(
    '''
        SELECT audit_task_id
        FROM audit_task
        WHERE user_id='%(user_id)s'
    ''' % locals(), connection)

def get_audit_interactions(audit_task_id):
    return pd.read_sql(
    '''
        SELECT *
        FROM audit_task_interaction
        WHERE audit_task_id='%(audit_task_id)s'
    ''' % locals(), connection)

In [40]:
action_types = pd.read_sql(
    '''
        SELECT DISTINCT action
        FROM audit_task_interaction
    ''', connection)
action_types = action_types['action'].tolist()
action_types

['ModeSwitch_Walk',
 'KeyboardShortcut_ModeSwitch_CurbRamp',
 'ViewControl_DoubleClick',
 'KeyboardShortcut_ClickOk',
 'KeyUp',
 'LowLevelEvent_mouseout',
 'LowLevelEvent_keydown',
 'LabelBeforeJump_Jump',
 'LabelBeforeJump_Blink',
 'KeyDown',
 'Redo_AddLabel',
 'Click_Redo',
 'ContextMenu_RadioChange',
 'KeyboardShortcut_ModeSwitch_NoSidewalk',
 'TaskStart',
 'Click_ZoomOut',
 'Click_Subcategory_Other',
 'LowLevelEvent_mousedown',
 'Onboarding_End',
 'TaskSubmit',
 'LowLevelEvent_mouseover',
 'ContextMenu_TextBoxFocus',
 'ModeSwitch_CurbRamp',
 'KeyboardShortcut_Severity_1',
 'Onboarding_Start',
 'ModeSwitch_Occlusion',
 'ContextMenu_TextBoxBlukr',
 'KeyboardShortcut_ZoomIn',
 'PopUpShow_GSVLabelDisappear',
 'PopUpMessage_SignInClick',
 'Click_ModeSwitch_NoCurbRamp',
 'LabelingCanvas_MouseUp',
 'Onboarding_Transition',
 'ModalComment_ClickOK',
 'ExplainThis_Click',
 'ContextMenu_Open',
 'Click_ModeSwitch_Obstacle',
 'KeyboardShortcut_Severity_3',
 'Click_ZoomIn',
 'Click_Subcategory_O

In [47]:
features_per_action = ['_total', '_per_pan_mean', '_per_pan_std']
action_features = [(action + feature) for action in action_types for feature in features_per_action ]
action_features.sort()
action_features

['Click_LabelDelete_per_pan_mean',
 'Click_LabelDelete_per_pan_std',
 'Click_LabelDelete_total',
 'Click_ModeSwitch_CurbRamp_per_pan_mean',
 'Click_ModeSwitch_CurbRamp_per_pan_std',
 'Click_ModeSwitch_CurbRamp_total',
 'Click_ModeSwitch_NoCurbRamp_per_pan_mean',
 'Click_ModeSwitch_NoCurbRamp_per_pan_std',
 'Click_ModeSwitch_NoCurbRamp_total',
 'Click_ModeSwitch_NoSidewalk_per_pan_mean',
 'Click_ModeSwitch_NoSidewalk_per_pan_std',
 'Click_ModeSwitch_NoSidewalk_total',
 'Click_ModeSwitch_Obstacle_per_pan_mean',
 'Click_ModeSwitch_Obstacle_per_pan_std',
 'Click_ModeSwitch_Obstacle_total',
 'Click_ModeSwitch_Other_per_pan_mean',
 'Click_ModeSwitch_Other_per_pan_std',
 'Click_ModeSwitch_Other_total',
 'Click_ModeSwitch_SurfaceProblem_per_pan_mean',
 'Click_ModeSwitch_SurfaceProblem_per_pan_std',
 'Click_ModeSwitch_SurfaceProblem_total',
 'Click_ModeSwitch_Walk_per_pan_mean',
 'Click_ModeSwitch_Walk_per_pan_std',
 'Click_ModeSwitch_Walk_total',
 'Click_OpenSkipWindow_per_pan_mean',
 'Click_O

In [44]:
def get_action_counts(audit_inter):
    features = {}
    
    for action in action_types:
        a = audit_inter[audit_inter['action'] == action]
        g = a.groupby('gsv_panorama_id').size()
        
        total = len(a)
        if len(a) == 0:
            mean = 0
            std = 0
        else: # TODO: handle stddev NaNs
            mean = g.mean()
            std = g.std()
                
            
        features[action + '_total'] = total
        features[action + '_per_pan_mean'] = mean
        features[action + '_per_pan_std'] = std
    
    return features

In [49]:
def get_features(user_id):
    all_audit_interactions = None
    
    feature_df = pd.DataFrame(columns=(['worker_id'] + action_features))

    audit_task_ids = get_audit_tasks(user_id)
    
    for audit_task_id in audit_task_ids.values:
        audit_interactions = get_audit_interactions(audit_task_id[0])
        
        if all_audit_interactions is None:
            all_audit_interactions = audit_interactions
        else:
            all_audit_interactions = all_audit_interactions.append(audit_interactions)
            
    if all_audit_interactions is None:
        logging.warning('SKIPPING worker_id because no iteraction: ' + str(user_id))
        return
    action_counts = get_action_counts(all_audit_interactions)
    feature_df.loc[len(feature_df)] = [user_id] + [action_counts[action_feature] for action_feature in action_features]
    return feature_df

In [50]:
features = None

for i, user in user_ids.iterrows():
    
    f = get_features(user['worker1'])
    if features is None:
        features = f
    else:
        features = features.append(f)

features.reset_index(inplace=True)

In [55]:
features

,index,worker_id,Click_LabelDelete_per_pan_mean,Click_LabelDelete_per_pan_std,Click_LabelDelete_total,Click_ModeSwitch_CurbRamp_per_pan_mean,Click_ModeSwitch_CurbRamp_per_pan_std,Click_ModeSwitch_CurbRamp_total,Click_ModeSwitch_NoCurbRamp_per_pan_mean,Click_ModeSwitch_NoCurbRamp_per_pan_std,...,ViewControl_DoubleClick_total,ViewControl_MouseDown_per_pan_mean,ViewControl_MouseDown_per_pan_std,ViewControl_MouseDown_total,ViewControl_MouseUp_per_pan_mean,ViewControl_MouseUp_per_pan_std,ViewControl_MouseUp_total,WalkTowards_per_pan_mean,WalkTowards_per_pan_std,WalkTowards_total
0,0,01232fef-5a19-4435-8be6-c0da3b38cabd,1,0,4,2.204545,1.249736,97,1.58824,0.618347,...,137,2.395200,2.899322,1497,2.358400,2.775153,1474,0,0,0
1,0,9501513f-3822-4921-861e-8f1440dee102,1,NaN,1,1.943925,1.294632,208,1.42553,0.71459,...,63,2.187234,1.966415,1028,2.170213,1.926354,1020,0,0,0
2,0,32f21407-253f-46ea-a01b-55bcf4ac2113,1.16667,0.408248,7,3.247059,2.092537,276,1.4,0.707107,...,33,3.468619,4.165605,1658,3.351464,4.002346,1602,0,0,0
3,0,b65c0864-7c3a-4ba7-953b-50743a2634f6,1.33333,0.57735,4,1.874172,1.318103,283,1.32353,0.53488,...,4,2.286598,2.967163,1109,2.255670,2.831456,1094,0,0,0
4,0,0bfed786-ce24-43f9-9c58-084ae82ad175,1,0,4,3.200000,2.388004,128,1.66667,0.57735,...,8,4.848684,6.557701,737,4.651316,6.087943,707,0,0,0
5,0,b4d389df-665e-4ccf-8951-56bbeba997e4,0,0,0,2.804878,1.873760,115,1.09091,0.301511,...,7,2.373253,2.836618,1189,2.361277,2.804856,1183,0,0,0
6,0,c744a3ea-aa20-4d40-89e4-878ca21c002e,1.15789,0.374634,22,2.177570,1.219442,233,1.6,0.894427,...,150,3.813121,4.233342,1918,3.727634,4.120613,1875,0,0,0
7,0,9931e187-5ab5-4376-bc39-b8da48600c40,1.71429,1.68379,24,3.169492,2.298069,187,2,1.07417,...,334,3.983087,5.424338,1884,3.826638,5.058510,1810,0,0,0
8,0,220da33e-7174-4b62-be1b-b7018dd16fab,1,0,4,2.707865,1.531504,241,1.25,0.46291,...,131,3.003891,4.231598,1544,2.881092,4.023793,1478,0,0,0
9,0,26e51ea3-440a-4730-ae04-1457aeeb686b,1.09091,0.301511,12,2.181818,1.470930,24,1.85714,1.06904,...,369,5.338658,5.250794,1671,5.156550,4.942402,1614,0,0,0


In [53]:
l = volunteer_accuracy
l = l[l['binary'] == True]
l = l[l['granularity'] == '5_meter']
l = l[l['remove.low.severity'] == False]
l = l[l['n.workers'] == 1]
labels = l[['condition.id', 'worker1', 'label.type', 'precision', 'recall', 'specificity', 'f.measure']]
labels.reset_index(inplace=True)
labels.head()

,index,condition.id,worker1,label.type,precision,recall,specificity,f.measure
0,0,70,01232fef-5a19-4435-8be6-c0da3b38cabd,Problem,0.5094339622641509,0.6136363636363636,0.8773584905660378,0.556701030927835
1,3,72,9501513f-3822-4921-861e-8f1440dee102,Problem,0.275,0.7586206896551724,0.7351598173515982,0.40366972477064217
2,6,73,32f21407-253f-46ea-a01b-55bcf4ac2113,Problem,0.4691358024691358,0.7916666666666666,0.7871287128712872,0.5891472868217054
3,9,74,b65c0864-7c3a-4ba7-953b-50743a2634f6,Problem,0.4126984126984127,0.8387096774193549,0.7175572519083969,0.553191489361702
4,12,75,0bfed786-ce24-43f9-9c58-084ae82ad175,Problem,0.3411764705882353,0.8285714285714286,0.7358490566037735,0.4833333333333334


In [56]:
data = labels.merge(features, left_on=['worker1'], right_on=['worker_id'])
data.head()

,index_x,condition.id,worker1,label.type,precision,recall,specificity,f.measure,index_y,worker_id,...,ViewControl_DoubleClick_total,ViewControl_MouseDown_per_pan_mean,ViewControl_MouseDown_per_pan_std,ViewControl_MouseDown_total,ViewControl_MouseUp_per_pan_mean,ViewControl_MouseUp_per_pan_std,ViewControl_MouseUp_total,WalkTowards_per_pan_mean,WalkTowards_per_pan_std,WalkTowards_total
0,0,70,01232fef-5a19-4435-8be6-c0da3b38cabd,Problem,0.5094339622641509,0.6136363636363636,0.8773584905660378,0.556701030927835,0,01232fef-5a19-4435-8be6-c0da3b38cabd,...,137,2.3952,2.899322,1497,2.3584,2.775153,1474,0,0,0
1,165,70,01232fef-5a19-4435-8be6-c0da3b38cabd,CurbRamp,0.6666666666666666,0.72,0.961038961038961,0.6923076923076923,0,01232fef-5a19-4435-8be6-c0da3b38cabd,...,137,2.3952,2.899322,1497,2.3584,2.775153,1474,0,0,0
2,166,70,01232fef-5a19-4435-8be6-c0da3b38cabd,NoCurbRamp,0,null,0.9609375,null,0,01232fef-5a19-4435-8be6-c0da3b38cabd,...,137,2.3952,2.899322,1497,2.3584,2.775153,1474,0,0,0
3,167,70,01232fef-5a19-4435-8be6-c0da3b38cabd,NoSidewalk,0.3225806451612903,0.5,0.9110169491525424,0.39215686274509803,0,01232fef-5a19-4435-8be6-c0da3b38cabd,...,137,2.3952,2.899322,1497,2.3584,2.775153,1474,0,0,0
4,168,70,01232fef-5a19-4435-8be6-c0da3b38cabd,Obstacle,0,0,0.9921568627450981,null,0,01232fef-5a19-4435-8be6-c0da3b38cabd,...,137,2.3952,2.899322,1497,2.3584,2.775153,1474,0,0,0


In [57]:
data.to_csv('volunteer_interaction_features.csv')